Three functional blocks

1. Transcriptomics, Proteomics, Active Gene IDs
2. Create or load tissue specific model
3. Disease Analysis: Differential Expressions
4. Knock out simulation: Score Genes, Generate Drug List

In [24]:
import sys
import os
import pandas
import numpy
import json
from subprocess import call
from project import configs
print(configs.rootdir)

/Users/zhzhao/Dropbox/Helikar/pipelines


## Step 1: Transcriptomics and Proteomics Generation

In [2]:
transcriptomics_config_file = 'GeneExpressionDataUsed.xlsx'
proteomics_data_file = 'ni.3693-S5.xlsx'
proteomics_supplementary_data_file = 'Supplementary Data 1.xlsx'

In [16]:
cmd = ' '.join(['python3', 'transcriptomic_gen.py', 
      '-i', '"{}"'.format(transcriptomics_config_file)])
!{cmd}

Input file is " GeneExpressionDataUsed.xlsx
---
Start Collecting Data for:
['GSE22886' 'GSE43005' 'GSE22045' 'GSE24634']
['GSM565273' 'GSM565274' 'GSM565275' 'GSM565290' 'GSM565291' 'GSM565292'
 'GSM1054773' 'GSM1054779' 'GSM1054781' 'GSM1054789' 'GSM548000'
 'GSM548001' 'GSM607510' 'GSM607511' 'GSM607512']
---

Initialize project (GSE22886):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565273.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565274.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565275.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565290.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565291.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE22886_RAW/GSM565292.tar
Retrieve Samples Complete

1924 single ENTREZ_GENE_IDs to merge
id_list: 5080, set: 3422
entrez_single_id_list: 22524, set: 22524
entrez_id_list: 1821, set: 1773
dups: 2568, set: 910
1253 id merged
Save to /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Th1.csv
---
Start Collecting Data for:
['GSE2770' 'GSE22886' 'GSE43769' 'GSE43005']
['GSM60357' 'GSM60358' 'GSM60362' 'GSM60368' 'GSM60373' 'GSM60379'
 'GSM60708' 'GSM60709' 'GSM60713' 'GSM60719' 'GSM60724' 'GSM60730'
 'GSM60742' 'GSM60743' 'GSM60747' 'GSM60753' 'GSM60758' 'GSM60764'
 'GSM565281' 'GSM565282' 'GSM565283' 'GSM565284' 'GSM565285' 'GSM565286'
 'GSM565998' 'GSM565999' 'GSM566000' 'GSM566001' 'GSM566002' 'GSM566003'
 'GSM1070756' 'GSM1070760' 'GSM1070764' 'GSM1054776' 'GSM1054784'
 'GSM1054788' 'GSM1054792']
---

Initialize project (GSE2770):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60357.tar
Sam

1831 single ENTREZ_GENE_IDs to merge
id_list: 4748, set: 3318
entrez_single_id_list: 22514, set: 22514
entrez_id_list: 1734, set: 1690
dups: 2256, set: 826
1234 id merged
Save to /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Th17.csv
---
Start Collecting Data for:
['GSE43005' 'GSE60679' 'GSE4527' 'GSE24634']
['GSM1054778' 'GSM1054786' 'GSM1054796' 'GSM1054794' 'GSM1484732'
 'GSM1484748' 'GSM1484756' 'GSM1484764' 'GSM1484733' 'GSM1484749'
 'GSM1484757' 'GSM1484765' 'GSM1484734' 'GSM1484750' 'GSM1484758'
 'GSM1484766' 'GSM1484735' 'GSM1484751' 'GSM1484759' 'GSM1484767'
 'GSM1484737' 'GSM1484739' 'GSM1484741' 'GSM1484743' 'GSM101519'
 'GSM101521' 'GSM607516' 'GSM607517' 'GSM607518' 'GSM607528' 'GSM607529'
 'GSM607530' 'GSM607540' 'GSM607541' 'GSM607542' 'GSM607552' 'GSM607553'
 'GSM607554']
---

Initialize project (GSE43005):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE43005_RAW
Sample exist: /Users/zhzhao/Dropbox/

In [22]:
cmd = ' '.join(['python3', 'proteomics_gen.py', 
      '-d', '"{}"'.format(proteomics_data_file), 
      '-s', '"{}"'.format(proteomics_supplementary_data_file)])
!{cmd}

Data file is " ni.3693-S5.xlsx
Supplementary Data file is " Supplementary Data 1.xlsx
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Naïve.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th1.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th2.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th17.csv


In [25]:
cmd = ' '.join(['python3', 'merge_xomics.py', 
      '-t', '"{}"'.format(transcriptomics_config_file), 
      '-p', '"{}"'.format(proteomics_supplementary_data_file)])
!{cmd}

Transcriptomics file is "GeneExpressionDataUsed.xlsx"
Proteomics file is "Supplementary Data 1.xlsx"
Read from /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Naive.csv
Read from /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Th1.csv
Read from /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Th2.csv
Read from /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Th17.csv
Read from /Users/zhzhao/Dropbox/Helikar/pipelines/data/transcriptomics_Treg.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Naïve.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th1.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th2.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th17.csv
(21850, 2)
(9150, 2)
699 single ENTREZ_GENE_IDs to merge
id_list: 3536, set: 3429
entrez_single_id_list: 21338, set: 21337
entrez_id_list: 1263, set: 1261
dups: 

## Step 1.5 Create Tissue Specific Model

In [26]:
step1_results_file = os.path.join(configs.datadir, 'step1_results_files.json')
with open(step1_results_file) as json_file:
    tissue_gene_exp = json.load(json_file)
print(tissue_gene_exp)

{'Th1': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/merged_Th1.csv', 'Naive': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/merged_Naive.csv', 'Th2': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/merged_Th2.csv', 'Th17': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/merged_Th17.csv', 'Treg': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/merged_Treg.csv'}


In [27]:
# (input) filename of General Model 
GeneralModelFile = 'GeneralModel.mat'
# (input) filename of Tissue Gene Expression
genefile = 'merged_Th1.csv'
# (output) filename of Tissue Specific Model
tissuefile = 'Th1_SpecificModel.mat'

In [28]:
tissue_spec_model = {}
for key,value in tissue_gene_exp.items():
    tissuefile = '{}_SpecificModel.mat'.format(key)
    tissue_spec_model[key] = tissuefile
    tissue_gene_file = value.split('/')[-1]
    tissue_gene_folder = os.path.join(configs.datadir, key)
    os.makedirs(tissue_gene_folder, exist_ok=False)
    cmd = ' '.join(['python3', 'create_tissue_specific_model.py', 
                      '-m', '"{}"'.format(GeneralModelFile), 
                      '-g', '"{}"'.format(tissue_gene_file),
                      '-o', '"{}"'.format(tissuefile)])
    !{cmd}

print(tissue_spec_model)

General Model file is " GeneralModel.mat
Gene Expression file is " merged_Th1.csv
Output file is " Th1_SpecificModel.mat
/Users/zhzhao/Dropbox/Helikar/pipelines
(20638, 2)
(1233, 2)
(3429, 2)
(24067, 2)
Map gene expression to reactions, 0 errors.
1's: 7582
2's: 0
Teff
Genes: 1108
Metabolites: 5507
Reactions: 7582
1.0*biomass_reaction_Mphage - 1.0*biomass_reaction_Mphage_reverse_6cff5
<Solution 0.000 at 0x18292b9e8>
General Model file is "GeneralModel.mat"
Gene Expression file is "merged_Naive.csv"
Output file is "Naive_SpecificModel.mat"
(20625, 2)
(1210, 2)
(3321, 2)
(23946, 2)
Map gene expression to reactions, 0 errors.
1's: 7494
2's: 0
Teff
Genes: 1069
Metabolites: 5457
Reactions: 7494
1.0*biomass_reaction_Mphage - 1.0*biomass_reaction_Mphage_reverse_6cff5
<Solution 0.000 at 0x17b43e6a0>
General Model file is "GeneralModel.mat"
Gene Expression file is "merged_Th2.csv"
Output file is "Th2_SpecificModel.mat"
(20544, 2)
(1231, 2)
(3428, 2)
(23972, 2)
Map gene expression to reactions, 0

## Step 2: Disease Gene Analysis
Differential Expression Analysis

Only 1 disease to be analyzed, output files in data folder

In [30]:
disease_gene_file = 'Disease_Gene_Analyzed.xlsx'

In [31]:
cmd = ' '.join(['python3', 'disease_analysis.py', 
              '-i', '"{}"'.format(disease_gene_file)])
!{cmd}

Input file is " Disease_Gene_Analyzed.xlsx
Initialize project (GSE21942):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066005.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066006.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066007.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066008.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066009.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066010.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066011.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066012.tar
Sample exist: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE21942_RAW/GSM1066013.tar
Sample exist: /Users/zhzhao/D

In [32]:
step2_results_file = os.path.join(configs.datadir, 'step2_results_files.json')
with open(step2_results_file) as json_file:
    disease_files = json.load(json_file)
print(disease_files)

{'GSE': 'GSE21942', 'UP_Reg': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/RA_UP_GSE21942.txt', 'DN_Reg': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/RA_DOWN_GSE21942.txt', 'RAW_Data': '/Users/zhzhao/Dropbox/Helikar/pipelines/data/Raw_Fit_GSE21942.csv'}


## Step 3: Knock Out Simulation
Knock out simulation and repurposing drug List generation

In [11]:
# datafolder = 'Th1'
# tissueSpecificModelfile = 'Th1_Cell_SpecificModel4manuscript.mat'
# inhibitors_file = 'Th1_inhibitors_Entrez.txt'

# RA_Down = 'RA_DOWN.txt'
# RA_Up = 'RA_UP.txt'
# drug_raw_file = 'Repurposing_Hub_export.txt'
# # drug_pre_file = 'Repurposing_Hub_Preproc.csv'


In [12]:
# cmd = ' '.join(['python3' , 'knock_out_simulation.py',
#               '-t', tissueSpecificModelfile,
#               '-i', inhibitors_file,
#               '-u', RA_Up,
#               '-d', RA_Down,
#               '-f', datafolder,
#               '-r', drug_raw_file])

In [35]:
tissue_spec_model

{'Th1': 'Th1_SpecificModel.mat',
 'Naive': 'Naive_SpecificModel.mat',
 'Th2': 'Th2_SpecificModel.mat',
 'Th17': 'Th17_SpecificModel.mat',
 'Treg': 'Treg_SpecificModel.mat'}

In [34]:
RA_Down = disease_files['DN_Reg']
RA_Up = disease_files['UP_Reg']

drug_raw_file = 'Repurposing_Hub_export.txt'

In [ ]:
for key,value in tissue_spec_model.items():
    tissueSpecificModelfile = value
    tissue_gene_folder = os.path.join(configs.datadir, key)
    os.makedirs(tissue_gene_folder, exist_ok=False)
    inhibitors_file = '{}_inhibitors_Entrez.txt'.format(key)
    cmd = ' '.join(['python3' , 'knock_out_simulation.py',
                  '-t', tissueSpecificModelfile,
                  '-i', inhibitors_file,
                  '-u', RA_Up,
                  '-d', RA_Down,
                  '-f', key,
                  '-r', drug_raw_file])
    !{cmd}
